In [1]:
# df_players
# name, elo, state (H/m), move time avg (not possible yet), acpl game avg
# df_games
# p1_ID, p2_ID, winner_ID, date, opening, p1_ELO, p2_ELO
# df_moves
# player_ID, game_ID, time, eval, fen/move, comment

In [2]:
import pandas as pd
import numpy as np
import uuid        # for id generation
import shortuuid

import chess.pgn
from stockfish import Stockfish

In [3]:
# stockfish = Stockfish(
#     'stockfish_14_linux_x64/stockfish_14_linux_x64/stockfish_14_x64', 
#     parameters={"Threads": 2, 'Min Split Depth': 26, 'Ponder':True}
# )
# stockfish.set_elo_rating(2600)
# stockfish.set_skill_level(30)

In [4]:
%%time

players = {
    'White':[],
    'White_Elo': [],
    'Black': [],
    'Black_Elo': [],
    'WhiteIsComp':[],
}

games = {
    'Game_ID': [],
    'Date' : [],
    'White':[],  # Dummy ID
    'White_Elo': [],
    'Black': [],  # Dummy ID
    'Black_Elo': [],
    'ECO': [],
    'Result': [],
}

moves_log_dict = {
    'Game_ID': [],
    'White': [],  # Dummy ID
    'Black': [],  # Dummy ID
    'ECO': [],
    'FEN_moves': [],
    #'cpl': [],
    'WhiteIsComp': [],
    'Result': [],
}

# read file
pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8')  # always a Comp vs Player
game_counter = 0

while True:  # keep reading games
    try:
        game = chess.pgn.read_game(pgn)
        board = game.board()
        moves = list(game.mainline_moves())
        
        # Player
        players['White_Elo'].append(game.headers['WhiteElo'])
        players['Black_Elo'].append(game.headers['BlackElo'])
        players['White'].append(game.headers['White'])
        players['Black'].append(game.headers['Black'])
        players['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        
        # Games
        games['Game_ID'].append(game.headers['FICSGamesDBGameNo'])
        games['White'].append(game.headers['White'])  # dummy ID
        games['Black'].append(game.headers['Black'])  # dummy ID
        games['White_Elo'].append(game.headers['WhiteElo'])
        games['Black_Elo'].append(game.headers['BlackElo'])
        games['ECO'].append(game.headers['ECO'])
        games['Result'].append(game.headers['Result'])
        games['Date'].append(game.headers['Date'])
        
        # Moves
        fen_pos = []
        acpl = []
        
        moves_log_dict['Game_ID'].append(game.headers['FICSGamesDBGameNo'])  # dummy ID for this dataset only breaks
        moves_log_dict['White'].append(game.headers['White'])  # dummy ID
        moves_log_dict['Black'].append(game.headers['Black'])  # dummy ID
        moves_log_dict['ECO'].append(game.headers['ECO'])
        moves_log_dict['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        moves_log_dict['Result'].append(game.headers['Result'])

        # MOVE CYCLE
        for move in moves:
            board.push(move)
            fen_pos.append(board.fen())
            #stockfish.set_fen_position(board.fen())  # load stockfish with current FEN for eval
#             cpl = stockfish.get_evaluation()['value']/100
#             acpl.append(cpl)
        
        moves_log_dict['FEN_moves'].append(fen_pos)
        #moves_log_dict['cpl'].append(acpl)
        
        game_counter += 1
        if game_counter == 55:  # number of games to read
            break
    except AttributeError:  # no further games to read
        print('No further games to load.')
        break

print(f'{game_counter} games read.')
#TODO takes ~1 sec to process 5 games, too slow.

55 games read.
CPU times: user 313 ms, sys: 1.07 ms, total: 314 ms
Wall time: 313 ms


In [5]:
#generates unique IDs from int
def id_generator(id):
    return uuid.uuid4().int

In [6]:
#generates unique IDs containing digits and characters
def short_id_gen(id):
    return shortuuid.ShortUUID().random(length=15)

## df_players

In [7]:
df_players = pd.DataFrame(players)
df_players.head()

,White,White_Elo,Black,Black_Elo,WhiteIsComp
0,forlat,1970,Geforce,2204,Yes
1,Geforce,2201,forlat,1973,No
2,forlat,1976,Geforce,2198,Yes
3,Geforce,2211,forlat,1963,No
4,forlat,1958,Geforce,2216,Yes


In [8]:
#pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8') 

In [9]:
#game = chess.pgn.read_game(pgn)

In [10]:
# def players_id_list(input_df, c_black, c_white): # c_white, c_black - columns of respective df, must be a string
#     black = input_df[c_black]
#     white = input_df[c_white]
#     players_names = list(set(list(black) + list(white)))
#     df_pl = {'Players': players_names}
#     players_id = pd.DataFrame(df_pl)
#     players_id['Player_ID'] = players_id['Players'].apply(id_generator)
#     return players_id

#download list? input_df.to_csv("/notebooks/data/players_id")

In [11]:
players_id = pd.DataFrame({'Players': [], 'Player_ID' : []})

def players_id_list(input_df, players_id):
    #extract black and white columns
    black = list(input_df["Black"]) 
    white = list(input_df["White"])
    
    #merge uniqe values from both columns:
    bw_merged = pd.DataFrame(list(set(black + white)), columns=["Players"])
    
    # Player_ID filled with NaNs:
    players_id = players_id.merge(bw_merged, how="outer", left_on=["Players"], right_on=["Players"])
    
    # NaNs replaced with generated IDs
    nans_to_ids = players_id["Player_ID"].fillna(players_id["Player_ID"].apply(id_generator))
    
    #inserting missing IDs to players_id
    players_id["Player_ID"] = nans_to_ids 
    return players_id

In [12]:
players_id = players_id_list(df_players, players_id)

In [13]:
players_id

,Player_ID,Players
0,154731496204555028951014948520934437175,Genobear
1,19415242707632665366224994185318906773,Qiyas
2,202676521458802706708938284972519436253,chenqisheng
3,113989065269182307371907526399435069525,Arsyah
4,162239632106608475814319096338507919120,chessloon
5,57712078342237451247443057150273450459,SpeckEngine
6,59682137456730448154987602434890577754,GriffySr
7,127548735801305660648060773155198186400,exeComp
8,147985404005677151159906536055965003074,IFDThor
9,103962209329530683575231133097968022122,nakshatra


In [14]:
df_dummy = pd.DataFrame({'White' : ["12345", 'DummyName', "1234", "forlat", "Geforce"], "Black" : ['DummyName', "12345", "Dummy", "Geforce", "Bambi"]})

In [15]:
df_dummy

,White,Black
0,12345,DummyName
1,DummyName,12345
2,1234,Dummy
3,forlat,Geforce
4,Geforce,Bambi


In [16]:
players_id  = players_id_list(df_dummy, players_id)

In [17]:
def assign_player_id(input_df): # returns a df with 2 new columns and assigned player ID
    #print("List od IDs has been generated:")
    #print(players_unique)
    m_white = input_df.merge(players_id, left_on=["White"], right_on=['Players'])   #
    m_white['White_ID'] = m_white['Player_ID']
    m_white.drop(columns=['Players', "Player_ID"], inplace=True)
    m_bw = m_white.merge(players_id, left_on=["Black"], right_on=['Players'])
    m_bw['Black_ID'] = m_bw['Player_ID']
    m_bw.drop(columns=['Players', "Player_ID"], inplace=True)
    df_players = m_bw
    return df_players

In [18]:
assign_player_id(df_dummy)

,White,Black,White_ID,Black_ID
0,12345,DummyName,24277665577541419211818004327186141557,6984531135465811163074988676114880777
1,DummyName,12345,6984531135465811163074988676114880777,24277665577541419211818004327186141557
2,1234,Dummy,261595572724435016682179234445817347338,327643169850463719285248267419626984268
3,forlat,Geforce,161782516147934208822491303210479251441,103097651153829821676832075847288016609
4,Geforce,Bambi,103097651153829821676832075847288016609,226116633567068595516611879982732826949


In [19]:
# Fix it:
def update_and_assign_player_IDs(input_df, players_id):
    df_players = assign_player_id(input_df)
    return df_players

In [20]:
update_and_assign_player_IDs(df_dummy, players_id)

,White,Black,White_ID,Black_ID
0,12345,DummyName,24277665577541419211818004327186141557,6984531135465811163074988676114880777
1,DummyName,12345,6984531135465811163074988676114880777,24277665577541419211818004327186141557
2,1234,Dummy,261595572724435016682179234445817347338,327643169850463719285248267419626984268
3,forlat,Geforce,161782516147934208822491303210479251441,103097651153829821676832075847288016609
4,Geforce,Bambi,103097651153829821676832075847288016609,226116633567068595516611879982732826949


## df_games

In [21]:
df_games = pd.DataFrame(games)
df_games['Game_ID'] = df_games['Game_ID'].apply(id_generator)
##TODO date to datetime
df_games

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result
0,160268984083736149335159601877233794287,2021.01.31,forlat,1970,Geforce,2204,A01,0-1
1,286315451540603702686726809536787920402,2021.01.31,Geforce,2201,forlat,1973,B00,1-0
2,190849447159600812744374317187540332701,2021.01.31,forlat,1976,Geforce,2198,A20,0-1
3,120519507389563520214121505333322495583,2021.01.31,Geforce,2211,forlat,1963,B10,0-1
4,51974105902504119209304291603670636429,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2
5,92180948032455383877304830292516443100,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0
6,58289846895725340186657045813873861326,2021.01.31,Geforce,2213,forlat,1961,B00,1-0
7,31364841005311982377214023647172396375,2021.01.31,forlat,1964,Geforce,2210,C28,0-1
8,305573341369701659114218011006790371397,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1
9,329903030753048846122018582727631752636,2021.01.31,Geforce,2207,forlat,1967,B10,1-0


In [22]:
timestamp = game.headers["Time"]

In [23]:
def game_id(df_games):
    df_games['Game_ID'] = df_games['Game_ID'].apply(short_id_gen)
    return df_games

In [24]:
game_id(df_games)

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result
0,7WNGH2z5BPdCoC4,2021.01.31,forlat,1970,Geforce,2204,A01,0-1
1,5xq52pUHHxmdf8J,2021.01.31,Geforce,2201,forlat,1973,B00,1-0
2,3nznj265cDeTYv7,2021.01.31,forlat,1976,Geforce,2198,A20,0-1
3,8A4qnopnWFoCpd6,2021.01.31,Geforce,2211,forlat,1963,B10,0-1
4,4cHm9MhvjMVnBxT,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2
5,3cLZsfdWbBcT2Gw,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0
6,Bub9asCEY8HEZft,2021.01.31,Geforce,2213,forlat,1961,B00,1-0
7,5iuTW4bQD6np7uo,2021.01.31,forlat,1964,Geforce,2210,C28,0-1
8,Bj8Q95WXCkJWDHC,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1
9,5uDnZa4vj2KjGuU,2021.01.31,Geforce,2207,forlat,1967,B10,1-0


In [61]:
def moves_id(id):
    for i in df_games.iterrows:
        i =+ 1
        return f'{df_games["Game_ID"][i]}-{df_games["Date"][i]}-{df_games["White"][i]}-{df_games["Black"][i]}'

In [62]:
len(df_games.index)

55

In [63]:
df_games['Moves_ID'] = ''

In [64]:
df_games['Moves_ID'] = df_games['Moves_ID'].apply(moves_id)

TypeError: 'method' object is not iterable

In [65]:
df_games

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result,Moves_ID
0,7WNGH2z5BPdCoC4,2021.01.31,forlat,1970,Geforce,2204,A01,0-1,
1,5xq52pUHHxmdf8J,2021.01.31,Geforce,2201,forlat,1973,B00,1-0,
2,3nznj265cDeTYv7,2021.01.31,forlat,1976,Geforce,2198,A20,0-1,
3,8A4qnopnWFoCpd6,2021.01.31,Geforce,2211,forlat,1963,B10,0-1,
4,4cHm9MhvjMVnBxT,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2,
5,3cLZsfdWbBcT2Gw,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0,
6,Bub9asCEY8HEZft,2021.01.31,Geforce,2213,forlat,1961,B00,1-0,
7,5iuTW4bQD6np7uo,2021.01.31,forlat,1964,Geforce,2210,C28,0-1,
8,Bj8Q95WXCkJWDHC,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1,
9,5uDnZa4vj2KjGuU,2021.01.31,Geforce,2207,forlat,1967,B10,1-0,


## df_moves

In [29]:
df_moves = pd.DataFrame(moves_log_dict)
#TODO moves and cpl as features when loading to DB
df_moves['Game_ID'] = df_games['Game_ID']
df_moves.head(20)

,Game_ID,White,Black,ECO,FEN_moves,WhiteIsComp,Result
0,7WNGH2z5BPdCoC4,forlat,Geforce,A01,[rnbqkbnr/pppppppp/8/8/8/1P6/P1PPPPPP/RNBQKBNR...,Yes,0-1
1,5xq52pUHHxmdf8J,Geforce,forlat,B00,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0
2,3nznj265cDeTYv7,forlat,Geforce,A20,[rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR...,Yes,0-1
3,8A4qnopnWFoCpd6,Geforce,forlat,B10,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,0-1
4,4cHm9MhvjMVnBxT,forlat,Geforce,D30,[rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR...,Yes,1/2-1/2
5,3cLZsfdWbBcT2Gw,exeComp,Ruvarashe,B92,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,Yes,1-0
6,Bub9asCEY8HEZft,Geforce,forlat,B00,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0
7,5iuTW4bQD6np7uo,forlat,Geforce,C28,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,Yes,0-1
8,Bj8Q95WXCkJWDHC,Ruvarashe,exeComp,C11,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,0-1
9,5uDnZa4vj2KjGuU,Geforce,forlat,B10,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0


# search

In [30]:
def search_df(df, column, value):
    try:
        index = np.where(df[column] == value)
        df = df.iloc[index]
        if len(df) == 0:
            return f'No games found for {value}.'
        return df
    except KeyError as e:
        print(e, f'not found.')